In [9]:
import gym

# Common imports
import numpy as np
import random
import os
import collections

# to make this notebook's output stable across runs
np.random.seed(42)


env = gym.make('LunarLander-v2')


# Best random sampling




In [10]:
#obs = []
def get_random_actions(env, seed):
    actions = []

    env.seed(seed)
    env.reset()

    step=0
    while True:
        step+=1
        action = env.action_space.sample()
        ob, rewards, terminated, info = env.step(action)
        actions.append(action)
        if terminated: break
    
    return actions

def get_random_rewards(env, seed):
    actions = []
    rewards = []

    env.seed(seed)
    env.reset()

    step=0
    while True:
        step+=1
        action = env.action_space.sample()
        ob, reward, terminated, info = env.step(action)
        actions.append(action)
        rewards.append(reward)
        if terminated: 
            break
    
    return rewards, actions


def explore_rewards(env, change_node, actions, old_rewards):
    env.seed(seed)
    env.reset()

    new_actions = []
    rewards = old_rewards[:-change_node]
    
    step=0
    for action in actions[:-change_node]:
        step+=1
        ob, reward, terminated, info = env.step(action)
        new_actions.append(action)
        if terminated: break

    if not terminated:
        # print("continue to explore", len(new_actions))
        while True:
            step+=1
            action = env.action_space.sample()
            ob, reward, terminated, info = env.step(action)
            new_actions.append(action), rewards.append(reward)
            if terminated: break

    return rewards, new_actions
    
def explore_actions(env, change_node, actions):
    env.seed(seed)
    env.reset()

    new_actions = []

    step=0
    for action in actions[:-change_node]:
        step+=1
        ob, rewards, terminated, info = env.step(action)
        new_actions.append(action)
        if terminated: break

    if not terminated:
        # print("continue to explore", len(new_actions))
        while True:
            step+=1
            action = env.action_space.sample()
            ob, rewards, terminated, info = env.step(action)
            new_actions.append(action)
            if terminated: break

    return new_actions


def find_best_rewards(n, env, actions, rewards):
    best_actions=[]
    best_rewards = []
    for ep in range(n):
        print(f" {ep} ".center(80, '*'))
        change_node=1
        # if len(actions) == 500:
        #     return actions
        if len(best_actions)>0:
            actions = best_actions
        if len(best_rewards)>1:
            rewards = best_rewards
        best_actions = []
        best_rewards = [-1e6] 
        while change_node<len(actions) and change_node < 100:
            new_rewards, new_actions = explore_rewards(env, change_node, actions, rewards)
            if sum(new_rewards)>sum(best_rewards):
                # print(len(new_actions), len(actions), change_node)
                #if len(new_actions)>len(best_actions):
                print(len(new_actions), len(actions), change_node, sum(new_rewards))
                best_actions=new_actions
                best_rewards=new_rewards
            change_node+=1
        
    return best_actions, best_rewards if len(best_actions)>len(actions) else actions


def get_obs(env, actions_result):
    obs_res = []
    for key in actions_result:
        env.seed(key)
        ob = env.reset()
        actions = actions_result[key]
        obs = []
        for action in actions:
            obs.append(ob)
            ob, rewards, terminated, info = env.step(action)
            
            if terminated: break
        
        obs_res.append(np.array(obs))
    
    return obs_res



In [11]:
seed=0
rewards, actions = get_random_rewards(env, seed)

new_r, new_act = explore_rewards(env, 50, actions, rewards)
# print(sum(new_r), sum(rewards))
best_actions, best_rewards = find_best_rewards(3, env, actions, rewards)

************************************** 0 ***************************************
102 102 1 -149.00837859088466
102 102 5 -141.56807267363666
102 102 16 -128.0574026152969
102 102 24 -125.55624193964569
104 102 28 -123.26043672630061
103 102 30 -115.39515325051048
124 102 32 -13.980617247773807
120 102 34 10.231381970022568
************************************** 1 ***************************************
120 120 1 10.231381970022568
120 120 4 11.686774316464266
1000 120 34 73.23397092383432
************************************** 2 ***************************************
1000 1000 1 73.60809107572727
1000 1000 3 75.35113808488286
1000 1000 9 77.36513234831155
1000 1000 13 77.63069625639572
1000 1000 16 78.33312204631495
1000 1000 59 78.48170719275846


In [13]:
import time
step=0
env = gym.make("LunarLander-v2")
env.seed(seed)
env.reset()

while True:
    # print(ob.reshape(1, -1).shape)
    action = best_actions[step]
    # action = res[seeds[4]][step]
    ob, reward, terminated, info = env.step(action)
    step+=1
    env.render()
    time.sleep(0.05)
    print(step)
    if terminated: break#steps.append(step);break

env.close()

KeyboardInterrupt: 

In [ ]:
res = {}
seeds = [0, 1, 5, 10, 21, 42, 47, 63, 84, 100, 121, 144]
for seed in seeds[:2]:
    actions = get_random_actions(env, seed)
    # best_actions = find_best_actions(env, actions)
    # print(f"[{seed}] Actions length improvement:",len(actions), "->", len(best_actions))
    # res[seed] = best_actions

NameError: name 'find_best_actions' is not defined

In [ ]:
obs = get_obs(env, res)
array_obs = np.concatenate(obs[:])
array_obs.shape

In [ ]:
array_actions = np.concatenate([np.array(act[1]) for act in res.items()][:])

# Plot data

In [ ]:
import matplotlib.pyplot as plt

color = array_actions
speed = array_obs[:, 3]
theta = array_obs[:, 2]
plt.figure(figsize=(7, 3), dpi=200)
plt.scatter(theta, speed, c=color, s=1), #print(obs[:, 2].shape)
plt.vlines([0.032, -0.032], ymin=min(speed), ymax=max(speed))
plt.hlines([0], xmin=min(theta), xmax=max(theta))
plt.xlabel(r'$\theta$')
plt.ylabel('omega')
plt.colorbar();

# Train LightGBM

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

lbc = LGBMClassifier(n_estimators=100)

lbc.fit(array_obs, array_actions)
accuracy_score(array_actions, lbc.predict(array_obs))

In [ ]:
lbc.feature_importances_

# Test model on environment
It suuucks, I saved *next_obs* and not the actual obs

In [ ]:
steps = []
for seed in range(1, 200, 5):
    env.seed(seed)
    ob = env.reset()

    step = 0
    while True:
        # print(ob.reshape(1, -1).shape)
        action = lbc.predict(ob.reshape(1, -1))[0]
        # action = res[seeds[4]][step]
        ob, reward, terminated, info = env.step(action)
        step+=1
        if terminated: steps.append(step);break
        
    print(seed, ':',step)

In [ ]:
np.median(steps), np.mean(steps), np.min(steps), np.max(steps)